### Scaling with Multiple Objects 


In [ ]:
%matplotlib inline

from IPython.display import HTML
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from ssp.maps import Spatial2D
from ssp.plots import heatmap_animation, create_gif
from ssp.dynamics import Trajectory
from functools import partial

In [ ]:
dim = 256
scale = 10

T = 2
dt = 0.05
n_trials = 5

x_len = 2
y_len = 2
x_spaces = 101
y_spaces = 101


ssp_map = Spatial2D(dim=dim, scale=scale)
ssp_map.build_grid(x_len, y_len, x_spaces, y_spaces)

In [ ]:
traj_a = Trajectory(T, dt)
traj_b = Trajectory(T, dt)

for sign, name in zip([1, -1], ['A', 'B']):
    dxdt_a = partial(lambda sign, t: sign * np.sin(t * np.pi), sign)
    dydt_a = partial(lambda sign, t: sign * np.cos(t * np.pi), sign)
    traj_a.add_object_spec(name, 1, 1, dxdt_a, dydt_a)

    dxdt_b = partial(lambda sign, t: sign * np.ones(len(t)), sign)
    dydt_b = lambda t: np.ones(len(t))
    traj_b.add_object_spec(name, 0 if sign > 0 else 2, 0, dxdt_b, dydt_b)


a_cue = ssp_map.initialize_cue(traj_a)
b_cue = ssp_map.initialize_cue(traj_b)

a_sims = []
for new_ssp in ssp_map.algebraic_dynamics_gen(traj_a):
    a_sims.append(ssp_map.compute_heatmap(new_ssp * a_cue))

b_sims = []
for new_ssp in ssp_map.algebraic_dynamics_gen(traj_b):
    b_sims.append(ssp_map.compute_heatmap(new_ssp * b_cue))

In [ ]:
# plot the results
figsize = (16, 8)
sims = [a_sims, b_sims]
titles = ['Trajectory A', 'Trajectory B']

ani = heatmap_animation(sims, figsize=figsize, titles=titles)
HTML('<img src="data:image/gif;base64,{0}" />'.format(create_gif(ani)))

### 2. Random Dynamics with Variable Numbers of Objects

Now, to examine the scaling of this approach, we can vary the number of objects encoded in the SSP and see how decoding accuracy changes. 

In [ ]:
def create_trajectory(n):
    '''Creates random dynamics for N distinct objects'''
    trajectory = Trajectory(T, dt)
    for idx in range(n):
        name = ssp_map.object_keys[idx]
        sign = np.random.choice([1, -1])
        
        # random initial object position
        x = np.random.uniform(0.5, x_len - 0.5)
        y = np.random.uniform(0.5, y_len - 0.5)
        
        # random oscillatory dynamics
        dxdt = lambda t: sign * np.random.random_sample() * np.sin(t * np.pi)
        dydt = lambda t: sign * np.random.random_sample() * np.sin(t * np.pi)
       
        trajectory.add_object_spec(name, x, y, dxdt, dydt)

    return trajectory


# generate dynamics with 2, 3, 4 encoded objects
sims = [[], [], []]
for i, n in enumerate(range(2, 5)):
    trajectory = create_trajectory(n)
    cue = ssp_map.initialize_cue(trajectory)
    
    for new_ssp in ssp_map.algebraic_dynamics_gen(trajectory):
        sims[i].append(ssp_map.compute_heatmap(new_ssp * cue))       
        
# plot the results
ani = heatmap_animation(sims, figsize=figsize)
HTML('<img src="data:image/gif;base64,{0}" />'.format(create_gif(ani)))

This obviously doesn't scale particularly well, since there's considerable noise with just four encoded objects. The reason for the poor scaling comes from the fact that the decoding cue used to extract the positions of each object is a sum 𝑁 terms (one per object), and each term creates 𝑁−1 noise terms when the decoding operation is performed. For an encoding with four objects, this means we have four terms of interest in the decoding, along with twelve noise terms. Because the decoding is a linear operation, it won't help to decode each object position in a sequence and combine the results. Rather, some kind of intermediate cleanup step is required. Otherwise, the number of noise terms scales as 𝑁2−𝑁, where 𝑁 is the number of encoded objects.


### 3. Random Dynamics with Intermediate Cleanup Steps

One way to reduce noise when decoding the positions of the objects from the SSP over time involves decoding each object's position seperately, cleaning up the result, and then combining these clean results to produce a visualization of the object's position.

In [ ]:
# generate dynamics with 5, 6, 7 encoded objects
sims = [[], [], []]
for i, n in enumerate(range(5, 8)):
    trajectory = create_trajectory(n)
    
    for new_ssp in ssp_map.algebraic_dynamics_gen(trajectory):
        decoded = ssp_map.cleanup(new_ssp, trajectory.object_names)
        sims[i].append(ssp_map.compute_heatmap(decoded))       

In [ ]:
#  plot the results
ani = heatmap_animation(sims, figsize=figsize)
HTML('<img src="data:image/gif;base64,{0}" />'.format(create_gif(ani)))

The results are much nicer here with respect to noise (the scaling is governed by the standard HRR trace capacity analysis), but the object trajectories are less smooth. The observed jittering occurs because the decoded position of each object is being "snapped" to the nearest point on the grid that defines how the heatmap is being computed. This could be smoothed by using an average over all items over the cleanup threshold rather than an argmax. Noise ocassionally "teleports" objects from one location another (this is unsurprising in the case with with 7 objects since our vector dimensionality is only 256).